# Strands SDK: Max Tokens Handling Explained

This notebook demonstrates how the Strands SDK handles the `max_tokens` limit when it's reached during model responses. We'll walk through the complete flow step-by-step.

## ✅ What You'll Learn

1. How to set up an agent with token limits
2. What happens when max_tokens is reached
3. The message recovery process
4. How the agent handles incomplete tool calls
5. Agent recovery after exceptions

## ❌ Key Behavior

When `max_tokens` is reached, Strands SDK **fails fast** rather than continuing with potentially corrupted responses.

## Setup and Imports

In [2]:
import logging
from strands import Agent, tool
from strands.models.bedrock import BedrockModel
from strands.types.exceptions import MaxTokensReachedException

# Enable logging to see internal recovery process
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Imports successful!")

✅ Imports successful!


## Define Tools

We'll create a tool that would typically generate a long response, making it likely to hit token limits.

In [3]:
@tool
def story_tool(story: str) -> str:
    """
    Tool that writes a story that is minimum 50,000 lines long.
    This tool is designed to trigger max_tokens in our example.
    """
    return story

@tool
def weather_tool(city: str) -> str:
    """Get weather information for a city."""
    return f"The weather in {city} is sunny and 75°F."

print("✅ Tools defined!")
print(f"📋 Available tools: story_tool, weather_tool")

✅ Tools defined!
📋 Available tools: story_tool, weather_tool


## Step 1: Create Agent with Low Token Limit

We'll create an agent with `max_tokens=100` - a very low limit that will easily be exceeded.

In [14]:
# Create model with very low token limit
model = BedrockModel(max_tokens=100)
agent = Agent(model=model, tools=[story_tool, weather_tool])

print(f"✅ Agent created!")
print(f"🔢 Token limit: {model.config.get('max_tokens')}")
print(f"🛠️ Available tools: {list(agent.tool_registry.registry.keys())}")
print(f"📊 Initial message count: {len(agent.messages)}")

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


✅ Agent created!
🔢 Token limit: 100
🛠️ Available tools: ['story_tool', 'weather_tool']
📊 Initial message count: 0


## Step 2: Trigger Max Tokens Exception

Let's make a request that will likely exceed our 100-token limit. The model will try to:
1. Generate a text response
2. Call the `story_tool` 
3. Get cut off mid-generation due to token limit

In [15]:
print("🚀 Making request: 'Tell me a story!'")
print("📝 Expected behavior:")
print("   1. Model starts generating response")
print("   2. Model attempts to call story_tool")
print("   3. Response gets truncated at 100 tokens")
print("   4. Tool call becomes incomplete")
print("   5. MaxTokensReachedException is raised")
print()

try:
    result = agent("Tell me a story!")
    print("❌ Unexpected: No exception was thrown!")
    print(f"Result: {result.stop_reason}")
    
except MaxTokensReachedException as e:
    print("✅ MaxTokensReachedException caught as expected!")
    # print(f"📄 Exception message: {str(e)}")
    # print()
    # print("🔍 This exception indicates:")
    # print("   - The model response was truncated")
    # print("   - Any tool calls were incomplete/corrupted")
    # print("   - The agent is in an unrecoverable state for this request")

🚀 Making request: 'Tell me a story!'
📝 Expected behavior:
   1. Model starts generating response
   2. Model attempts to call story_tool
   3. Response gets truncated at 100 tokens
   4. Tool call becomes incomplete
   5. MaxTokensReachedException is raised

I'd be happy to tell you a story! Let me create one for you.
Tool #1: story_tool


INFO:strands.event_loop._recover_message_on_max_tokens_reached:handling max_tokens stop reason - replacing all tool uses with error messages


✅ MaxTokensReachedException caught as expected!


## Step 3: Examine Message Recovery

Before the exception was thrown, the SDK automatically recovered the message by:
1. Preserving all text content
2. Replacing incomplete tool calls with error messages
3. Saving the cleaned message to conversation history

In [16]:
print("📋 Conversation History After Max Tokens:")
print(f"📊 Total messages: {len(agent.messages)}")
print()

for i, message in enumerate(agent.messages):
    print(f"💬 Message {i+1} ({message['role']}):")
    
    for j, content in enumerate(message.get('content', [])):
        if 'text' in content:
            text = content['text']
            # Truncate long text for display
            # display_text = text if len(text) <= 150 else text[:150] + "..."
            display_text = text
            print(f"   📄 Text content {j+1}: {display_text}")
            
        elif 'toolUse' in content:
            tool_use = content['toolUse']
            print(f"   🔧 Tool use {j+1}: {tool_use}")
            
    print()

📋 Conversation History After Max Tokens:
📊 Total messages: 2

💬 Message 1 (user):
   📄 Text content 1: Tell me a story!

💬 Message 2 (assistant):
   📄 Text content 1: I'd be happy to tell you a story! Let me create one for you.
   📄 Text content 2: The selected tool story_tool's tool use was incomplete due to maximum token limits being reached.



In [12]:
# # Inspect the complete conversation flow
# def inspect_message_flow(messages):
#     print("=== DETAILED MESSAGE FLOW ===")
    
#     for i, message in enumerate(messages):
#         print(f"\n--- Message {i+1} ---")
#         print(f"Role: {message['role']}")
        
#         for j, content in enumerate(message['content']):
#             print(f"  Content {j+1}:")
            
#             if 'text' in content:
#                 text = content['text']
#                 # Truncate long text for readability
#                 # if len(text) > 200:
#                 #     text = text[:200] + "..."
#                 print(f"    Text: {text}")
            
#             elif 'toolUse' in content:
#                 tool_use = content['toolUse']
#                 print(f"    Tool Use: {tool_use['name']}")
#                 print(f"    Input: {tool_use['input']}")
#                 print(f"    ID: {tool_use['toolUseId']}")
            
#             elif 'toolResult' in content:
#                 tool_result = content['toolResult']
#                 print(f"    Tool Result: {tool_result['status']}")
#                 print(f"    ID: {tool_result['toolUseId']}")
#                 # Don't print full content as it's very long
#                 print(f"    Content: [Raw KB Response - {len(str(tool_result['content']))} chars]")

# # Run the inspection
# inspect_message_flow(agent.messages)


In [17]:
agent.messages

[{'role': 'user', 'content': [{'text': 'Tell me a story!'}]},
 {'content': [{'text': "I'd be happy to tell you a story! Let me create one for you."},
   {'text': "The selected tool story_tool's tool use was incomplete due to maximum token limits being reached."}],
  'role': 'assistant'}]

## Step 4: Verify Recovery Error Message

Let's check if the expected recovery error message was inserted by the SDK.

In [18]:
# Look for the specific error message that indicates tool recovery
expected_error_text = "tool use was incomplete due to maximum token limits being reached"

# Extract all text content from messages
all_text_content = [
    content["text"]
    for message in agent.messages
    for content in message.get("content", [])
    if "text" in content
]

# Check if recovery message exists
has_recovery_message = any(expected_error_text in text for text in all_text_content)

print(f"🔍 Recovery Process Verification:")
print(f"   Expected error text: '{expected_error_text}'")
print(f"   ✅ Recovery message found: {has_recovery_message}")
print()

if has_recovery_message:
    print("💡 This confirms the SDK successfully:")
    print("   1. Detected incomplete tool use")
    print("   2. Replaced it with an informative error message")
    print("   3. Preserved the conversation context")
else:
    print("❌ Recovery message not found - unexpected behavior")

🔍 Recovery Process Verification:
   Expected error text: 'tool use was incomplete due to maximum token limits being reached'
   ✅ Recovery message found: True

💡 This confirms the SDK successfully:
   1. Detected incomplete tool use
   2. Replaced it with an informative error message
   3. Preserved the conversation context


## Step 5: Test Agent Recovery

Even though we hit max_tokens, the agent should still be functional for new requests. Let's test this by:
1. Removing tools to avoid tool-related token usage
2. Making a simple request

In [21]:
print("🔄 Testing Agent Recovery:")
print("   Removing tools to avoid tool-related token usage...")

# Clear tools to prevent tool use in recovery test
original_registry = agent.tool_registry.registry.copy()
original_config = agent.tool_registry.tool_config.copy() if agent.tool_registry.tool_config is not None else None

agent.tool_registry.registry = {}
agent.tool_registry.tool_config = {}

print(f"   🛠️ Tools cleared. Remaining tools: {list(agent.tool_registry.registry.keys())}")
print()

try:
    print("🧮 Making simple request: 'What is 3+3?'")
    recovery_result = agent("What is 3+3?")
    
    print("✅ Recovery successful!")
    print(f"📊 Stop reason: {recovery_result.stop_reason}")
    print(f"📄 Response: {recovery_result.message['content'][0]['text']}")
    print()
    print("💡 This demonstrates:")
    print("   - Agent remains functional after MaxTokensReachedException")
    print("   - Previous conversation history is preserved")
    print("   - New requests are processed normally")
    
except Exception as recovery_error:
    print(f"❌ Recovery failed: {recovery_error}")
    print("This would indicate a more serious issue with agent state")

🔄 Testing Agent Recovery:
   Removing tools to avoid tool-related token usage...
   🛠️ Tools cleared. Remaining tools: []

🧮 Making simple request: 'What is 3+3?'
3 + 3 = 6✅ Recovery successful!
📊 Stop reason: end_turn
📄 Response: 3 + 3 = 6

💡 This demonstrates:
   - Agent remains functional after MaxTokensReachedException
   - Previous conversation history is preserved
   - New requests are processed normally


## Step 6: Final Conversation State

Let's examine the complete conversation after recovery to see how everything was preserved.

In [22]:
print("📋 Final Conversation State:")
print(f"📊 Total messages: {len(agent.messages)}")
print()

for i, message in enumerate(agent.messages):
    print(f"💬 Message {i+1} ({message['role']}):")
    
    for j, content in enumerate(message.get('content', [])):
        if 'text' in content:
            text = content['text']
            # Show first and last part of long text
            if len(text) > 100:
                display_text = text[:50] + "..." + text[-50:]
            else:
                display_text = text
            print(f"   📄 {display_text}")
            
    print()

# Restore tools for future use
agent.tool_registry.registry = original_registry
agent.tool_registry.tool_config = original_config
print(f"🛠️ Tools restored: {list(agent.tool_registry.registry.keys())}")

📋 Final Conversation State:
📊 Total messages: 4

💬 Message 1 (user):
   📄 Tell me a story!

💬 Message 2 (assistant):
   📄 I'd be happy to tell you a story! Let me create one for you.
   📄 The selected tool story_tool's tool use was incomplete due to maximum token limits being reached.

💬 Message 3 (user):
   📄 What is 3+3?

💬 Message 4 (assistant):
   📄 3 + 3 = 6

🛠️ Tools restored: ['story_tool', 'weather_tool']


## Summary: How Strands SDK Handles Max Tokens

### 🔄 The Complete Flow:

1. **Request Processing** (`src/strands/agent/agent.py:377`)
   - Agent receives user prompt
   - Calls `event_loop_cycle()` to process request

2. **Model Invocation** (`src/strands/event_loop/event_loop.py:144`)
   - Model generates response with `max_tokens=100` limit
   - Response gets truncated when limit is reached
   - Returns `stop_reason="max_tokens"`

3. **Message Recovery** (`src/strands/event_loop/event_loop.py:162`)
   - `recover_message_on_max_tokens_reached()` is called
   - Preserves all text content
   - Replaces incomplete tool calls with error messages
   - Returns cleaned message

4. **Exception Handling** (`src/strands/event_loop/event_loop.py:221`)
   - Cleaned message is added to conversation history
   - `MaxTokensReachedException` is raised
   - Agent state is preserved but request terminates

### ✅ What's Preserved:
- Conversation history with cleaned messages
- Agent configuration and state
- Text content from model responses

### ❌ What's Prevented:
- Execution of incomplete/corrupted tool calls
- Continuation with potentially invalid state
- Silent failures or unexpected behavior

### 🔧 Design Philosophy:
**Fail Fast, Preserve State** - Rather than attempting to continue with potentially corrupted data, the SDK fails immediately while preserving conversation context for recovery.

## Advanced Example: Multiple Tool Calls

Let's see what happens when max_tokens is reached during multiple tool calls.

In [ ]:
# Create a fresh agent for this test
model_multi = BedrockModel(max_tokens=150)  # Slightly higher to see partial progress
agent_multi = Agent(model=model_multi, tools=[story_tool, weather_tool])

print("🔬 Advanced Test: Multiple Tool Calls with Max Tokens")
print(f"🔢 Token limit: {model_multi.config.get('max_tokens')}")
print()

try:
    # Request that might trigger multiple tool calls
    result = agent_multi("Tell me a story about the weather in New York, then get the actual weather there")
    print("❌ No exception - request completed within token limit")
    print(f"Stop reason: {result.stop_reason}")
    
except MaxTokensReachedException as e:
    print("✅ MaxTokensReachedException during multi-tool scenario")
    print()
    
    # Analyze what tool calls were affected
    tool_recovery_messages = []
    for message in agent_multi.messages:
        for content in message.get('content', []):
            if 'text' in content and 'tool use was incomplete' in content['text']:
                tool_recovery_messages.append(content['text'])
    
    print(f"🔧 Tool recovery messages found: {len(tool_recovery_messages)}")
    for i, msg in enumerate(tool_recovery_messages):
        print(f"   {i+1}. {msg}")

## Key Takeaways

### ✅ **Do:**
- Set appropriate `max_tokens` for your use case
- Handle `MaxTokensReachedException` in your application
- Trust that conversation state is preserved
- Continue using the agent after exceptions

### ❌ **Don't:**
- Assume tool calls will complete when near token limits
- Ignore `MaxTokensReachedException` 
- Set `max_tokens` too low for complex tasks
- Expect partial tool execution

### 🎯 **Best Practices:**
1. **Monitor token usage** in your applications
2. **Set generous limits** for tool-heavy workflows
3. **Implement retry logic** with higher limits if needed
4. **Log exceptions** for debugging and optimization

The Strands SDK's approach ensures **reliability and predictability** even when hitting resource limits.